# Sports Team Ranking
## Data Processing
This notebook is used for data exploration and for processing the data that will be analyzed. <br/>
<br/>
Originally coded by Rhys-Jasper León <br/>
Last updated 2025 November 10